In [ ]:
# Convert Greenland radiostratigraphy v2 individual campaign .mat files into GeoPackages with traced reflections for each traced segment
# 
# Joe MacGregor (NASA/GSFC)
# Last updated: 19 December 2024

In [1]:
# import necessary packages
import geopandas as gpd
import glob
import mat73
import numpy as np
import pandas as pd
from shapely.geometry import Point
import warnings

# ignore fragmentation warnings during dataframe generation
warnings.filterwarnings('ignore')

In [2]:
# paths to use
path_mat = '/Users/jamacgre/OneDrive - NASA/research/matlab/pickgui_v2/mat/breakout/mat/'
path_gpkg = '/Users/jamacgre/OneDrive - NASA/research/matlab/pickgui_v2/mat/breakout/gpkg/'

# examine .mat path
dir_mat = sorted(glob.glob(path_mat + '*.mat'))
num_mat = len(dir_mat)

# loop through each campaign .mat file
for i, file_mat in enumerate(dir_mat):
    
    mat_curr = mat73.loadmat(file_mat) # load current campaign's .mat file
    
    print(str(i) + ': ' + mat_curr['campaign'])
    
    # loop through each segment in current campaign
    for j in range(mat_curr['num_segment'].astype(int)):
        
        print(str(i) + ': ' + str(j) + ': ' + mat_curr['segment']['name'][j])
        
        # generate dataframe for current segment
        if (mat_curr['num_segment'].astype(int) == 1): # deal with situation where only 1 traced segment in that campaign
            df_curr = pd.DataFrame({'X':mat_curr['segment']['x'], 'Y':mat_curr['segment']['y']})
        else:
            df_curr = pd.DataFrame({'X':mat_curr['segment']['x'][j], 'Y':mat_curr['segment']['y'][j]})
        
        # add more metadata fields
        if (mat_curr['num_segment'].astype(int) == 1):
            # df_curr['firn_corr'] = mat_curr['segment']['firn_corr']
            df_curr['thick'] = mat_curr['segment']['thick']
            df_curr['time'] = mat_curr['segment']['time']            
        else:
            # df_curr['firn_corr'] = mat_curr['segment']['firn_corr'][j]
            df_curr['thick'] = mat_curr['segment']['thick'][j]
            df_curr['time'] = mat_curr['segment']['time'][j]
        
        # loop through each layer and add its properties, this is not pretty, repeats scalars unnecessarily (age and age_uncert), and generates a lot of fragmentation warnings, but it works
        # NOTE age and age_uncert variables are recorded in metadata that are saved in a more compact format, but these are currently very difficult to read using any package
        if (mat_curr['num_segment'].astype(int) == 1): # again only 1 traced segment
            for k in range(mat_curr['segment']['num_layer'].astype(int)):
                df_curr['depth' + str(k)] = mat_curr['segment']['stratigraphy'][k]['depth']                
                # df_curr['age' + str(k)] = mat_curr['segment']['stratigraphy'][k]['age']
                # df_curr['age_uncert' + str(k)] = mat_curr['segment']['stratigraphy'][k]['age_uncert']
                # df_curr['elev' + str(k)] = mat_curr['segment']['stratigraphy'][k]['elev']
                # df_curr['int' + str(k)] = mat_curr['segment']['stratigraphy'][k]['int']
                # df_curr['twtt' + str(k)] = mat_curr['segment']['stratigraphy'][k]['twtt']
        else:
            if (mat_curr['segment']['num_layer'][j].astype(int) == 1):
                    df_curr['depth' + str(1)] = mat_curr['segment']['stratigraphy'][j]['depth']                
                    # df_curr['age' + str(1)] = mat_curr['segment']['stratigraphy'][j]['age']
                    # df_curr['age_uncert' + str(1)] = mat_curr['segment']['stratigraphy'][j]['age_uncert']
                    # df_curr['elev' + str(1)] = mat_curr['segment']['stratigraphy'][j]['elev']
                    # df_curr['int' + str(1)] = mat_curr['segment']['stratigraphy'][j]['int']
                    # df_curr['twtt' + str(1)] = mat_curr['segment']['stratigraphy'][j]['twtt']
            else:
                for k in range(mat_curr['segment']['num_layer'][j].astype(int)):
                    df_curr['depth' + str(k)] = mat_curr['segment']['stratigraphy'][j]['depth'][k]                    
                    # df_curr['age' + str(k)] = mat_curr['segment']['stratigraphy'][j]['age'][k]
                    # df_curr['age_uncert' + str(k)] = mat_curr['segment']['stratigraphy'][j]['age_uncert'][k]
                    # df_curr['elev' + str(k)] = mat_curr['segment']['stratigraphy'][j]['elev'][k]
                    # df_curr['int' + str(k)] = mat_curr['segment']['stratigraphy'][j]['int'][k]
                    # df_curr['twtt' + str(k)] = mat_curr['segment']['stratigraphy'][j]['twtt'][k]
        
        # defragment after the fact
        df_curr = df_curr.copy()
        
        # generate geodataframe for current segment and assign to correct projection
        gdf_curr = gpd.GeoDataFrame(df_curr.drop(columns=['X', 'Y']), geometry=gpd.points_from_xy(df_curr.X, df_curr.Y), crs='epsg:3413')
        
        # metadata dictionary with reflection ages and uncertainties
        if (mat_curr['num_segment'].astype(int) == 1):
            age_dict = dict(zip([('age' + str(age)) for age in range(mat_curr['segment']['num_layer'].astype(int))] + 
                                [('age_uncert' + str(age)) for age in range(mat_curr['segment']['num_layer'].astype(int))], 
                                set(map(str, np.concatenate((np.array([mat_curr['segment']['stratigraphy'][age]['age'] for age in range(mat_curr['segment']['num_layer'].astype(int))]), 
                                                             np.array([mat_curr['segment']['stratigraphy'][age]['age_uncert'] for age in range(mat_curr['segment']['num_layer'].astype(int))])))))))
        else:
            if (mat_curr['segment']['num_layer'][j].astype(int) == 1):
                age_dict = dict(zip([('age' + str(1))] + [('age_uncert' + str(1))], 
                                    set(map(str, np.concatenate((np.ravel(mat_curr['segment']['stratigraphy'][j]['age']), np.ravel(mat_curr['segment']['stratigraphy'][j]['age_uncert'])))))))                
            else:
                age_dict = dict(zip([('age' + str(age)) for age in range(mat_curr['segment']['num_layer'][j].astype(int))] + 
                                    [('age_uncert' + str(age)) for age in range(mat_curr['segment']['num_layer'][j].astype(int))], 
                                    set(map(str, np.concatenate((mat_curr['segment']['stratigraphy'][j]['age'], mat_curr['segment']['stratigraphy'][j]['age_uncert']))))))
        
        # save resulting gpkg
        if (mat_curr['num_segment'].astype(int) == 1):
            gdf_curr.to_file((path_gpkg + mat_curr['segment']['name'] + '.gpkg'), driver='GPKG', metadata=age_dict)
        else:
            try:
                gdf_curr.to_file((path_gpkg + mat_curr['segment']['name'][j] + '.gpkg'), driver='GPKG', metadata=age_dict)
            except:
                # gdf_curr.drop(gdf_curr.filter(regex='age').columns, axis=1) # get rid of age columns because of size limitations, let someone figure them out from the the metadata
                gdf_curr.to_file((path_gpkg + mat_curr['segment']['name'][j] + '.gpkg'), driver='GPKG', metadata=age_dict)

0: 1993_Greenland_P3
0: 0: 19930624_01_009-012
0: 1: 19930702_01_011-013
0: 2: 19930703_01_007-009
0: 3: 19930703_01_010-012
1: 1995_Greenland_P3
1: 0: 19950518_01_001-005
1: 1: 19950518_01_006-007
1: 2: 19950518_01_013-015
1: 3: 19950522_01_005-006
1: 4: 19950522_01_008-010
1: 5: 19950522_01_011-016
1: 6: 19950522_01_017-018
1: 7: 19950523_01_021-025
1: 8: 19950524_01_007-008
1: 9: 19950524_01_021-023
2: 1996_Greenland_P3
2: 0: 1
3: 1997_Greenland_P3
3: 0: 19970511_01_003-008
3: 1: 19970511_01_009-009
3: 2: 19970511_01_010-010
3: 3: 19970511_01_011-011
3: 4: 19970517_01_009-010
3: 5: 19970517_01_012-014
3: 6: 19970519_01_009-010
3: 7: 19970519_01_012-012
3: 8: 19970519_01_013-013
3: 9: 19970521_01_008-011
3: 10: 19970521_01_013-013
3: 11: 19970521_01_015-015
3: 12: 19970521_01_020-021
3: 13: 19970523_01_005-011
3: 14: 19970524_01_001-001
3: 15: 19970524_01_002-002
3: 16: 19970524_01_016-017
3: 17: 19970524_01_018-019
3: 18: 19970527_01_001-004
3: 19: 19970527_01_008-011
3: 20: 1997052